In [11]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests
import lxml

In [12]:
regex_bbc = r"(http(s?):\/\/(www)?[A-Za-z0-9\/\.]+)"
regex_cbc = r"(http(s?):\/\/(www)?[A-Za-z0-9-?=\s\/\.]+)"
xpath_bbc = ['//div[@class="story-body__inner"]/p/text()',
            '//ul/li[@class="tags-list__tags"]/a/text()']

# Regex "&amp"
regex_decode = r"(\&amp)"

In [22]:
file_bbc = './datasets/bbchealth.txt'
file_cbc = './datasets/cbchealth.txt'
columns_name = ['tweetid','date','title','backup']

df = pd.read_csv(file_cbc, sep='|', header= None,error_bad_lines=False)
df.columns = columns_name
df.head()

b'Skipping line 846: expected 3 fields, saw 4\nSkipping line 904: expected 3 fields, saw 4\nSkipping line 914: expected 3 fields, saw 4\nSkipping line 1264: expected 3 fields, saw 4\nSkipping line 1269: expected 3 fields, saw 4\nSkipping line 1293: expected 3 fields, saw 4\nSkipping line 1348: expected 3 fields, saw 4\nSkipping line 1430: expected 3 fields, saw 4\nSkipping line 1486: expected 3 fields, saw 4\nSkipping line 1710: expected 3 fields, saw 4\nSkipping line 2699: expected 3 fields, saw 4\nSkipping line 2728: expected 3 fields, saw 4\nSkipping line 3000: expected 3 fields, saw 4\n'


ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [23]:
# Split Url out of content
def get_url(regex, sample):
    result = re.search(regex, sample)[0]
    
    return result

# Split text out of content
def split_txt_form_url(regex, sample):
    url = get_url(regex, sample)
    result = sample.strip(url)
    
    return result

# Scrapt content base on xpath command.
def scrapt_w_xpath(xpath_cmd):
    result = dict()
    
    # Content
    contents = xpath_selector.xpath(xpath_cmd[0]).extract()
    result.update({'Contents': contents})
    
    # Tags
    tags = xpath_selector.xpath(xpath_cmd[1]).extract()
    result.update({'Tags': tags})
    
    return result

In [35]:
title_attempt = True

def fetch_contents(target_url, target_df, list_xpath):
    for key, url in enumerate(target_url):

        # Start the connection
        res = requests.get(url)

        # Initialize XPath
        xpath_selector = Selector(text=res.text)

        # Get return content as text
        text_html = res.text

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
    #         soup = BeautifulSoup(res.content, 'lxml')
            xpath_selector = Selector(text=text_html)

            # Scrapt content with XPath
            list_scrapt = scrapt_w_xpath(xpath_bbc)

            count_xp_scrapt = len(list_scrapt)


            if count_xp_scrapt:

                # Add new columns
                if title_attempt:
                    df_temp = pd.DataFrame(columns = list(list_scrapt.keys()))
                    target_df = pd.concat([bbc, df_temp])
                    print(target_df.columns)
                    title_attempt = False

                # Add to dataFrame
                for k_, v_ in list_scrapt.items():
                    target_df[k_].iloc[key] = v_

            else:
                print("Return list is emptied ! ({})".format(url))
        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))

In [39]:
cbc = pd.DataFrame()
# cbc content
## //p/text()
## //div[@class='detailSummary']/text() --> Detail Summary

## //h2[@class='deck']/text() --> h2-sub-title
## //div[@class='story']/span --> Content
## //div[@class='source']/p/text() --> Author
## //div[@class="bylineDetails"]/span[@class="authorText"] --> Author


# list_xpath = ['//div[@class="story"]/span',
#             '//div[@class="source"]/p/text()']

# test_url = ['http://www.cbc.ca/news/canada/hamilton/news/hamilton-police-send-mental-health-pros-to-the-front-lines-with-cops-1.3024000?cmp=rss']

# fetch_contents(test_url,cbc, list_xpath)

In [53]:
url = 'https://www.cbc.ca/news/canada/hamilton/news/hamilton-police-send-mental-health-pros-to-the-front-lines-with-cops-1.3024000?cmp=rss'
xpath_author = '//div[@class="bylineDetails"]/span[@class="authorText"]/span/text()'
                
xpath_content = '//div[@class="story"]/span/p/text()'
# Start the connection
res = requests.get(url)

if res.status_code == 200:
    # Initialize XPath
    xpath_selector = Selector(text=res.text)

    # Get return content as text
    text_html = res.text

    xpath_selector = Selector(text=text_html)

    author = xpath_selector.xpath(xpath_author).extract()
    # https://stackoverflow.com/questions/34246574/how-to-get-text-from-inside-a-span-class-classnametext-i-want-span-in-pyth/34246790
    print(author)

    content = xpath_selector.xpath(xpath_content).extract()
#     print(content)
    
    # Join list of content into 1 simple paragraph
    seperator = ', '
    print(seperator.join(content))
else:
    print("[ERROR] at: ", res.status_code)

['Kelly Bennett']
Beginning Sunday, whenever a 9-1-1 call might involve a mental health situation, Hamilton police will be able to send a special cruiser with more than just an officer inside. Alongside the officer responding to that call will be a mental health worker., And they will tackle that call as a team., The service's innovative Mobile Crisis Rapid Response Team program, which sends mental health professionals from St. Joseph's Healthcare Hamilton out on 9-1-1 calls with Hamilton police officers expands to seven days a week starting Sunday., Based on a successful 15-month pilot project, the program aims to provide new means of support to community members in mental distress, as well as to cut down on emergency room time for both the people exhibiting mental illness symptoms and the uniformed cops., The idea, in part, is that with a mental health worker in the car along with the cop, situations can be approached from multiple, not just law enforcement, perspectives., "We do it 